In [1]:
import numpy as np
import tensorflow as tf

### 1. Concept of Autoencoders 

* Autoencoders are simple neural networks that their output is their input. 

<img src="https://cdn-images-1.medium.com/max/1600/0*2B-ZLwJTsi21avTO.jpg" width = 500 />

from [source](https://towardsdatascience.com/how-to-generate-images-using-autoencoders-acfbc6c3555e)

* Applications:
    * [Recommendation System](https://towardsdatascience.com/deep-autoencoders-for-collaborative-filtering-6cf8d25bbf1d)
    
    * Feature Extraction
    * Non Linear Dimensionality Reduction
    * Image Recognition


### 2. Creat Input Data

In [2]:
d = [1,0,0,0,0,0,0,0]
np.roll(d,0)

array([1, 0, 0, 0, 0, 0, 0, 0])

In [3]:
input_data = []
for i in range(8):
    input_data.append(d)
    d = np.roll(d, 1)
input_data = np.array(input_data).astype(np.float32)

In [4]:
input_data.shape, input_data.dtype

((8, 8), dtype('float32'))

In [5]:
input_data

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### 3. Build Autoencoder - Tensorflow

##### 1. Create Some Constants

 - here 9 dimensions will be compressed to 3 dimensions
 - activation function is predefined as sigmoid

In [42]:
num_input = 8
num_hid = 3
num_output = num_input
lr = 0.01
actf = tf.nn.sigmoid

##### 2. Network Configuration

![autoencoder](pic/autoencod838.jpg)

 - Define its layer with its weights and bias variables
 - and activation functions

In [44]:
input_layer = tf.placeholder(dtype = tf.float32, shape = [None, num_inputs])

hid_layer_param = {
'w': tf.Variable(tf.random_normal([num_input, num_hid])),
'b': tf.Variable(tf.zeros([num_hid]))}

output_layer_param = {
'w': tf.Variable(tf.random_normal([num_hid, num_output])),
'b': tf.Variable(tf.zeros([num_output]))}

hid_layer = actf(input_layer @ hid_layer_param['w'] + hid_layer_param['b'])
output_layer = actf(hid_layer @ output_layer_param['w'] + output_layer_param['b'])


##### 3. Define Loss Function, Accuracy Calculation

In [45]:
loss = tf.reduce_mean(tf.square(output_layer - input_layer))

optimizer = tf.train.AdamOptimizer(lr)
train = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(input_layer, axis = 1),
                              tf.argmax(output_layer, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         

##### 4. Training

In [46]:
def autoencoder(num_epoch):
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epoch):
            _, c, acc = sess.run([train, loss, accuracy], feed_dict = {input_layer: input_data})
            
            if epoch % 100 == 0:
                print('epoch {} ------- loss {} ------- accuracy {}'.format(epoch, c, acc)) 
                                
    return hid_layer_param, output_layer_param


In [49]:
num_epoch = 1000
autoencoder(num_epoch)

epoch 0 ------- loss 0.33663737773895264 ------- accuracy 0.125
epoch 100 ------- loss 0.10662546753883362 ------- accuracy 0.125
epoch 200 ------- loss 0.08875894546508789 ------- accuracy 0.75
epoch 300 ------- loss 0.07639152556657791 ------- accuracy 0.75
epoch 400 ------- loss 0.06549400091171265 ------- accuracy 0.75
epoch 500 ------- loss 0.056102100759744644 ------- accuracy 0.875
epoch 600 ------- loss 0.048292048275470734 ------- accuracy 0.875
epoch 700 ------- loss 0.04183731973171234 ------- accuracy 0.875
epoch 800 ------- loss 0.036529265344142914 ------- accuracy 1.0
epoch 900 ------- loss 0.03229247033596039 ------- accuracy 1.0


({'w': <tf.Variable 'Variable_8:0' shape=(8, 3) dtype=float32_ref>,
  'b': <tf.Variable 'Variable_9:0' shape=(3,) dtype=float32_ref>},
 {'w': <tf.Variable 'Variable_10:0' shape=(3, 8) dtype=float32_ref>,
  'b': <tf.Variable 'Variable_11:0' shape=(8,) dtype=float32_ref>})

### 4. Build Autoencoder - From Scratch

In [320]:
class Autoencoder:
    
    def __init__(self):
        self.num_input = 8 + 1
        self.num_output = self.num_input
        self.num_hid = 3
        self.w1 = np.random.randn(self.num_input, self.num_hid)          
        self.w2 = np.random.randn(self.num_hid, self.num_output)
        
    def sigmoid(self, x):
        # Activation function used to map any real value between 0 and 1
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, sig):
        # Use chain rule to calculate the derivatie of sigmoid function
        return sig *(sig - 1)
        
    def forwardfeed(self, x):
        net1 = x @ self.w1
        a1 = self.sigmoid(net1)
        net2 = a1 @ self.w2
        a2 = self.sigmoid(net2) 
        return a1, a2
    
    def backprop(self, lr, a1, a2):
        output_layer_derivative = (self.x - a2) * self.sigmoid_derivative(a2)
        hid_layer_derivative = output_layer_derivative @ self.w2.T * self.sigmoid_derivative(a1)
        
        self.w1 -= lr * self.x.T @ hid_layer_derivative 
        self.w2 -= lr * a1.T @ output_layer_derivative
        
    def reshape_x(self, x):
        return np.column_stack((np.full(x.shape[0],1), x))   
    
    def get_loss(self, a2):
        loss = 1 / self.x.shape[0] * np.sum(1/2 * (self.x - a2)**2)
        return loss
    
    def get_accuracy(self, a2):
        acc_count = 0
        for i in range(self.x.shape[0]):
            if np.argmax(self.x[i]) == np.argmax(a2[i]):
                acc_count += 1
        return acc_count/self.x.shape[0]
    
    def train(self, x, num_epoch, lr):  
        self.x = self.reshape_x(x)
        
        for epoch in range(num_epoch):
            a1, a2 = self.forwardfeed(self.x)
            self.backprop(lr, a1, a2)
            loss = self.get_loss(a2)
            accuracy = self.get_accuracy(a2)
            if epoch % 100 == 0:
                print('Epoch {} - Loss {} ------- Accuracy {}'.format(epoch, loss, accuracy))
                
        return a1
                               

In [327]:
autoencoder = Autoencoder()

In [328]:
autoencoder.train(input_data, num_epoch = 1000, lr = 0.001)

Epoch 0 - Loss 0.9513842450085215 ------- Accuracy 0.375
Epoch 100 - Loss 0.9250220438222978 ------- Accuracy 0.5
Epoch 200 - Loss 0.9008933694265226 ------- Accuracy 0.625
Epoch 300 - Loss 0.8789218425348546 ------- Accuracy 0.75
Epoch 400 - Loss 0.8590171045234578 ------- Accuracy 1.0
Epoch 500 - Loss 0.8410795733790483 ------- Accuracy 1.0
Epoch 600 - Loss 0.8250042812748747 ------- Accuracy 1.0
Epoch 700 - Loss 0.810683933696456 ------- Accuracy 1.0
Epoch 800 - Loss 0.7980113052295594 ------- Accuracy 1.0
Epoch 900 - Loss 0.7868810651553684 ------- Accuracy 1.0


array([[0.57136523, 0.52657692, 0.30672693],
       [0.21209031, 0.78467793, 0.24089505],
       [0.31615934, 0.18286173, 0.55534662],
       [0.38408309, 0.6386491 , 0.12080812],
       [0.60207807, 0.87219061, 0.31911204],
       [0.1661541 , 0.82280513, 0.64028918],
       [0.03775574, 0.3696642 , 0.1939201 ],
       [0.09857713, 0.5314371 , 0.42924396]])